Here is what I have done/learnt/know are the new problems 

In [ ]:
import numpy as np
from scipy.ndimage import maximum_filter, gaussian_filter, uniform_filter
import time 
from scipy import signal
import matplotlib.pyplot as plt
import dask as da
import dask_image.ndfilters
import zarr

#Params
hpf_width = 100 # order of 100
sdnorm_exp = 1 # range 0.8-1
npil_filt = gaussian_filter
npil_size = (1.0, 5.0, 5.0)
cv_filt = uniform_filter
cv_size = (1, 2, 2) # in dask needs to be int
int_thresh = 0

In [ ]:
# clean and commented basic function, This is the non-parallelised but working  function

#intput data is 4-D array(t,z,y,x)

def minimal_corrmap_base(movx,):
    """Input a 4-D array (t,z,y,x) """
    nt,nz,ny,nx = movx.shape

    #basic highpass filter
    mov_hpf = np.zeros_like(movx)
    for i in range(0, nt, hpf_width):
        mov_hpf[i:i+hpf_width] = movx[i:i+hpf_width] - movx[i:i+hpf_width].mean(axis=0) # subtracts mean of a width

    del movx
    sdmov = (((np.diff(mov_hpf, axis=0) ** 2).sum(axis=0)) / nt)**0.5 #standard deviation of each array point through time
    sdmov = np.greater(10 ** -9, sdmov)
    norm_mov = mov_hpf / (sdmov ** sdnorm_exp) #normalised the HPF movie
    del mov_hpf
    

    c1 = npil_filt(np.ones((nz,ny,nx)), sigma = npil_size, mode='constant')
    #c2 = cv_filt(np.ones((nz,ny,nx)), cv_size, mode='constant')

    #each time step apply the neuropil filter
    np_mov = np.zeros_like(norm_mov) 
    for i in range(len(norm_mov[0])):
        np_mov[i] = npil_filt(norm_mov[i], sigma = npil_size, mode='constant') / c1


    np_sub_mov = norm_mov - np_mov # removes the neuropil from the normalised image
    del norm_mov
    del np_mov

    
    #applies a cell detection filter
    cv_mov = np.zeros_like(np_sub_mov)
    for i in range(len(np_sub_mov[0])):
        cv_mov[i] = cv_filt(np_sub_mov[i], cv_size, mode='constant') * cv_size[-1] #should this be c2

    #removes values below a threshold defult = 0, and sum over time
    vmap = ((cv_mov ** 2) * (cv_mov > int_thresh).astype(int)).sum(axis=0) ** 0.5

    return vmap

In [ ]:
### There may be away to explicity store parts to disk or use .persist cleverly to eliminate the disk spill, personally I dont knowhow without spening more time learning about dask
### The cuurent problem seems to be down to I/O, and is exasterbated if input array is big (~ a sizebale fraction of the RAM (1/4 ~1/2))
### The problem maybe that one(+) of the functions aren't lazy, and them being called is a slow down


def minimal_corrmap_dask(movx):
    """Input 4-D dask array form (t,z,y,x)"""
    nt,nz,ny,nx = movx.shape

    # Here is naive rechunk based of the number of cores I have and to chunk so chunks are [100MB-1GB]
    # spatial_chunks = (-1,nz, ny//12, nx//12)
    # time_chunks = (10,nz,ny,nx)
    # here was ~ the optimal for my set up and data set (100,30,800,500)
    spatial_chunks = (-1,nz,200,150)
    time_chunks = (5,nz,ny,nx)
    

    #basic highpass filter
    movx.rechunk(spatial_chunks)
    mov_hpf = da.array.zeros_like(movx)
    for i in range(0, nt, hpf_width):
        mov_hpf[i:i+hpf_width] = movx[i:i+hpf_width] - movx[i:i+hpf_width].mean(axis=0) # subtracts mean of a width

    sdmov = (((da.array.diff(mov_hpf, axis=0) ** 2).sum(axis=0)) / nt)**0.5  #standard deviation of each array point through time
    sdmov = da.array.maximum(10 ** -9,sdmov) 

    norm_mov = mov_hpf / (sdmov ** sdnorm_exp) #normalised the HPF movie .. probably current error

    del mov_hpf
    del sdmov 

    norm_mov.rechunk(time_chunks)

    c1 = dask_image.ndfilters.gaussian_filter(da.array.ones((nz,ny,nx)), sigma = npil_size, mode='constant')
    #c2 = dask_image.ndfilters.uniform_filter(da.array.ones((nz,ny,nx)), cv_size, mode='constant')

    #each time step apply the neuropil filter
    np_mov = da.array.zeros_like(movx)
    for i in range(len(movx[0])):
        np_mov[i] = dask_image.ndfilters.gaussian_filter(norm_mov[i], sigma = npil_size, mode='constant') / c1


    np_sub_mov = norm_mov - np_mov # removes the neuropil from the normalised image

    del np_mov
    del norm_mov

    #applies a cell detection filter
    cv_mov = da.array.zeros_like(movx)
    for i in range(len(movx[0])):
        cv_mov[i] = dask_image.ndfilters.uniform_filter(np_sub_mov[i], cv_size, mode='constant') * cv_size[-1]

    del np_sub_mov

    #removes values below a threshold defult = 0, and sum over time
    cv_mov.rechunk(spatial_chunks)
    vmap = (((cv_mov ** 2) * (cv_mov > int_thresh).astype(int)).sum(axis=0) ** 0.5)

    return vmap


I have been stroing and loading as zarr e.g data_dask = da.array.from_zarr('test_data_larger.zarr')

test_data_larger, is sliced in  x,y (1256,972) -> (800,500)  to make it managable on my 16GB of RAM

On this data the base function ~ 196 seconds, 145 seconds (ran 2 times)

and the dask function ~ 37 seconds, 36 seconds (ran 2 times )

If data of the input is a few time less than RAM, the spepd is significantly quicker as spilling to the disk is the bottle neck, (These following numbers arnt 100% accurate just to give an idea) The input data for the (100,30,1256,972) is ~ 11GB as zarr, therfore RAM of ~> 40GB should be able to run it without spilling, therfore much quicker.

The dask function *should* be able to generalise to larger than ram data sets however this will bring significant slowdown from spilling to disk and sometimes has not been able to run.
There may be some memory leak, i havent gotten around to testing it/learning how to, however I belive there are other process on my laptop which could influence this.